## Задание 

Представьте, что вы работаете в крупном дейтинговом приложении.

Помимо базовых функций, в приложении также имеется премиум-подписка, которая дает доступ к ряду важных дополнительных возможностей. Был проведен A/B тест, в рамках которого для новых пользователей из нескольких стран была изменена стоимость премиум-подписки* при покупке через две новые платежные системы. При этом стоимость пробного периода оставалась прежней.

Проверьте: Был ли эксперимент успешен в целом.

*Деньги за подписку списываются ежемесячно до тех пор, пока пользователь её не отменит.

Всего есть три группы: тестовая (test), контрольная 1 (control_1) и контрольная 2 (control_2). Для каждой из них:

users_*.csv – информация о пользователях

transactions_*.csv – информация о платежах пользователей

#### Общие размышления 

Мы имеем три группы, две из которых являются контрольными. Две контрольные группы могут использоваться для того, чтобы определить, что мы разбили наших пользователей равноценно и они не отличаются по каким-либо показателям. К тому же время выравнивания показателей в контрольных группах поможет в будущем предполагать время, которое будет занимать эксперимент.

Опишем процедуру проведения a/b теста в случае двух контрольных групп:

1) Формируем гипотезу.

2) Определяем ключевую метрику(и).

3) Делим пользователей на группы.

4) Проводим a/a тестирование( определяем, что пользователи в контрольных группах не отличаются по тем показателям, которые не являются ключевыми )

5) Выбираем стат. тест для проверки ключевых метрик

6) Проводим эксперимент на всех трех группах( контрольные группы можно объединить, если будет показано, что они не отличаются по не ключевым показателям)

7) На основания результатов теста делаем вывод о результатах эксперимента

В данном случае тест уже проведен. Пункт 1,3 и 6 уже выполнены. Наша задача - показать, что 3 и остальные пункты были проведены корректно, выбрать самостоятельно метрики и, смотря на метрики, определить результат эксперимента.

Сейчас мы выступаем на стороне бизнеса, поэтому оценивать результат будем с точки зрения бизнеса. Сама суть бизнеса в том, чтобы зарабатывать деньги и, в теории, все его действия направлены на то, чтобы поддерживать и увеличивать финансовые показатели прямо или косвенно. 

В условии сказано, что стоимость подписки была изменена( направление изменения не указано). На что в поведении пользователей могло влиять изменение цены подписки? При этом важно, что данное поведение должно быть полезным бизнесу(судя по описанию данных не только покупатели подписки приносят бизнесу деньги).

1) Больше(меньше) пользователей могли приобретать подписку после введения изменения. Похоже на конверсию.

2) Теперь платящий за подписку пользователь в среднем приносит больше(меньше) денег.

3) Косвенным образом меняется количество денег в среднем на пользователя

Показатель 2 - arppu, а 3 - arpu. Быть может мы добавим еще метрики, если увидим внутри приложения взаимосвязь премиум-подписки с другими показателями.

Итак, общее направление действий определено. Теперь стоит приступить к изучению таблиц и взаимосвязей в них

### Импорт библиотек

In [2]:
import pandas as pd
import requests as r
import pingouin as pg

### Считывание файлов 


In [3]:
# нам предстоит проделать одинаковую операцию(считывания файла) 6 раз, поэтому будет лучше написать функцию.
# Функция принимает ссылку на файл и параметр, который сообщает, есть ли в файле колонки с типом дата(по умолчанию нет)
def reading_file(link, have_dates=0):
    file = r.get(f'https://cloud-api.yandex.net/v1/disk/public/resources?public_key={link}').json()['file']
    
    if have_dates:
        df = pd.read_csv(file, sep=';', parse_dates=[2, 3])
    else:
        df = pd.read_csv(file, sep=';')
        # здесь разделитель ; - это связано с особенностью данных в одной из колонок
        
    return df

In [4]:
users_test = reading_file('https://disk.yandex.ru/d/4XXIME4osGrMRA')
users_control_1 = reading_file('https://disk.yandex.ru/d/yJFydMNNGkEKfg')
users_control_2 = reading_file('https://disk.yandex.ru/d/br6KkQupzzTGoQ')
transactions_test = reading_file('https://disk.yandex.ru/d/gvCWpZ55ODzs2g', 1)
transactions_control_1 = reading_file('https://disk.yandex.ru/d/VY5W0keMX5TZBQ', 1)
transactions_control_2 = reading_file('https://disk.yandex.ru/d/th5GL0mGOc-qzg', 1)

# Разведочный анализ данных

По условию, мы видим, что файлы о трех группах организованы схожим образом. Было бы удобно изучить информацию о них разом.

### Файлы users 

In [5]:
users_test.head()

,uid,age,attraction_coeff,coins,country,visit_days,gender,age_filter_start,age_filter_end,views_count,was_premium,is_premium,total_revenue
0,892309896,27,685,NaN,United States of America,"1,2,3,4,5,6,7,8,9,10,11,12,16,17",1,24,30,89,NaN,NaN,0
1,892044516,27,0,NaN,Germany,NaN,1,24,30,0,NaN,NaN,0
2,892185708,45,44,NaN,Israel,"1,3,4,6,7,8,10,11,12,13,14,15,16,17,18,19,20,2...",1,42,48,68,NaN,NaN,0
3,892130292,32,0,NaN,United States of America,"1,2,3,4,5,6,7,8,9,10,11,12",1,29,35,0,NaN,NaN,0
4,891406035,27,1000,NaN,France,NaN,1,24,30,1,1.0,NaN,0


Кроме того, мы знаем, что изменение вводилось на новых пользователях, в некоторых странах, стоимость подписки менялась при покупке через 2 новые платежные системы. Из всей этой информации здесь можно оценить только информацию о странах.
Также мы можем в первом приближении посмотреть на количество премиум-клиентов и выручку в каждой группе.

In [6]:
"""Функция для выведения информации о группе пользователей
На вход подается таблица с данными о пользователях и название этой таблицы,
чтобы потом отобразить его при выведении информации"""

def data_tale(df, df_name):
    result = pd.DataFrame([[df.shape[0]],
                           [df.columns[df.isna().any()].tolist()],
                           [df.uid.nunique()], 
                           [df.duplicated().sum()],
                           [df.country.nunique()],
                           [df.was_premium.sum()],
                           [df.is_premium.sum()],
                           [df.age.min()],
                           [df.age.max()],
                           [df.age.mean()],
                           [df.total_revenue.mean()],
                           [df.total_revenue.sum()]],
                          index=['Number_of_users', 
                                 'Columns_with_gaps', 
                                 'Unique_users', 
                                 'Duplicate_lines', 
                                 'Unique_countries', 
                                 'Number_of_was_premium', 
                                 'Number_of_is_premium', 
                                 'Min_age', 
                                 'Max_age', 
                                 'Average_age', 
                                 'Average_revenue', 
                                 'Sum_revenue'], 
                          columns=[df_name])
                                                         
    return result

In [7]:
""""Для каждой из таблиц, мы обращаемся к функции, а потом все 3 полученных столбца соединяем в один 
датафрейм, в котором можно удобно оценить показатели всех 3 таблиц"""

users_data_tale = data_tale(users_test, 'test')
users_data_tale['control_1'] = data_tale(users_control_1, 'control_1')
users_data_tale['control_2'] = data_tale(users_control_2, 'control_2')
users_data_tale['explanation'] = pd.DataFrame([['Количество записей'],
                                               ['Колонки с пропусками'],
                                               ['Уникальные пользователи'],
                                               ['Повторяющиеся строки'],
                                               ['Страны в этой группе'],
                                               ['Количество was_premium'],
                                               ['Количество is_premium'],
                                               ['Минимальный возраст'],
                                               ['Максимальный возраст'],
                                               ['Средний возраст'],
                                               ['Средняя выручка'],
                                               ['Общая выручка']],
                                              index=['Number_of_users', 
                                                     'Columns_with_gaps', 
                                                     'Unique_users', 
                                                     'Duplicate_lines', 
                                                     'Unique_countries', 
                                                     'Number_of_was_premium', 
                                                     'Number_of_is_premium', 
                                                     'Min_age', 
                                                     'Max_age', 
                                                     'Average_age', 
                                                     'Average_revenue', 
                                                     'Sum_revenue'])

users_data_tale

,test,control_1,control_2,explanation
Number_of_users,4308,4340,4264,Количество записей
Columns_with_gaps,"[coins, visit_days, was_premium, is_premium]","[coins, visit_days, was_premium, is_premium]","[coins, visit_days, was_premium, is_premium]",Колонки с пропусками
Unique_users,4308,4340,4264,Уникальные пользователи
Duplicate_lines,0,0,0,Повторяющиеся строки
Unique_countries,51,40,45,Страны в этой группе
Number_of_was_premium,408,436,411,Количество was_premium
Number_of_is_premium,157,192,191,Количество is_premium
Min_age,16,16,16,Минимальный возраст
Max_age,99,99,99,Максимальный возраст
Average_age,31.8893,32.0954,32.0462,Средний возраст


#### Выводы по сводной таблице users 

Внутри каждой таблицы пользователи уникальны и их количество различается не более чем на 80 человек при среднем числе пользователей 4304.

Во всех 3 таблицах одни и те же столбцы содержат пропущенные значения, а дубликатов нет.

Количество стран в группах разнится, но на данный момент - это не беда. Позднее нужно будет сделать список стран единообразным, чтобы следовать концепции эксперимента.

Количество тех, кто был премиум различается. При среднем 418.3 количество пользователей в тестовой группе различается максимально на 28 человек, что составляет 7% от среднего числа таких пользователей среди всех групп.

Для тех, кто является премиум ситуация отличается. При среднем 180, количество пользователей в тестовой группе отличается максимально на 35 человек, что составляет 19% от среднего количества.

Минимальный, максимальный возраст пользователей одинаковы, средний отличиается, но различие не кажется значительным.

Средняя выручка на пользователя и общая выручка различаются во всех группах

In [8]:
# Соберем информацию о типах данных в таблицах о пользователях и представим ее в одном месте
users_dtypes = users_test.dtypes.to_frame().rename(columns={0: 'users_test'})
users_dtypes['users_control_1'] = users_control_1.dtypes
users_dtypes['users_control_2'] = users_control_2.dtypes

users_dtypes

,users_test,users_control_1,users_control_2
uid,int64,int64,int64
age,int64,int64,int64
attraction_coeff,int64,int64,int64
coins,float64,float64,float64
country,object,object,object
visit_days,object,object,object
gender,int64,int64,int64
age_filter_start,int64,int64,int64
age_filter_end,int64,int64,int64
views_count,int64,int64,int64


Количество колонок небольшое и мы можем визуально оценить, что типы данных в каждой из таблиц совпадают.

Сейчас я вижу два возможных варианта продолжения исследования данных:

1) Подробнее заняться исследованием колонок, заполнение пропусков, взаимосвязями между колонками и выручкой, а потом перейти к поэтапному исследованию таблиц с транзакциями

2) Сделать первичный обзор transactions и их типов данных, а потом приступать к заполнению пропусков и поисков взаимосвязей в внутри users и transactions, а также между ними.

В данном случае я выберу второй вариант, так как природа пропусков и взаимосвязей с общей выручкой может раскрыться, если посмотреть файлы с записями о выручке с каждого конкретного пользователя

### Файлы transactions 

In [9]:
transactions_test.head()

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
0,891345942,Italy,2017-05-11 13:00:00,2017-11-13 15:04:00,12909,147,trial_vip_popup,trial_premium
1,892054251,United States of America,2017-10-22 00:33:00,2017-10-30 01:37:00,13923,147,trial_vip_popup,trial_premium
2,892236423,United States of America,2017-10-18 01:09:00,2017-10-23 00:15:00,3783,67,menu,other_type
3,892236423,United States of America,2017-10-18 01:09:00,2017-10-27 22:38:00,3783,67,menu,other_type
4,892168170,United States of America,2017-10-19 17:10:00,2017-10-27 19:10:00,9087,147,trial_vip_popup,trial_premium


В этих таблицах есть информация о дате регистрации и датах платежей пользователей( думаю, поможет нам определить понятие нового пользователя);

также есть информация о странах, что точнее позволит нам определить список стран, в которых были введены изменения;

есть информация об идентификаторе платежа( возможно, это позволит выйти на платежные системы и те 2 из них, которые новые);

revenue в совокупности с product_type позволит нам оценить стоимость новой подписки; 

In [10]:
# Функция для выведения информации о транзакциях

def data_tale_transactions(df, df_name):
    result = pd.DataFrame([[df.shape[0]],
                           [df.uid.nunique()],
                           [df.columns[df.isna().any()].tolist()],
                           [df.duplicated().sum()],
                           [df.country.nunique()],
                           [df.joined_at.min().strftime("%Y-%m-%d")],
                           [df.joined_at.max().strftime("%Y-%m-%d")],
                           [df.paid_at.min().strftime("%Y-%m-%d")],
                           [df.paid_at.max().strftime("%Y-%m-%d")],
                           [(df.joined_at >= df.paid_at).any()],
                           [df.query('revenue <= 0').revenue.any()],
                           [df.revenue.sum()],
                           [df.payment_id.nunique()],
                           [df.product_type.nunique()]], 
                          index=['Number_of_transactions', 
                                 'Unique_users', 
                                 'Columns_with_gaps', 
                                 'Duplicate_lines', 
                                 'Unique_countries', 
                                 'Min_joined_date', 
                                 'Max_joined_date', 
                                 'Min_paid_date', 
                                 'Max_paid_date', 
                                 'Error_in_dates', 
                                 'Not_positive_revenue', 
                                 'Sum_revenue', 
                                 'Number_of_ids', 
                                 'Number_of_products'], 
                          columns=[df_name])
                                                         
    return result

In [11]:
"""Для каждой из таблиц, мы обращаемся к функции, а потом все 3 полученных столбца соединяем в один 
датафрейм, в котором можно удобно оценить все показатели """

transactions_data_tale = data_tale_transactions(transactions_test, 'test')
transactions_data_tale['control_1'] = data_tale_transactions(transactions_control_1, 'control_1')
transactions_data_tale['control_2'] = data_tale_transactions(transactions_control_2, 'control_2')
transactions_data_tale['explanation'] = pd.DataFrame([['Размер таблицы'], 
                                                      ['Уникальные пользователи'], 
                                                      ['Колонки с пропусками'], 
                                                      ['Повторяющиеся строки'], 
                                                      ['Страны'], 
                                                      ['Минимальная дата регистрации'], 
                                                      ['Последняя дата регистрации'], 
                                                      ['Минимальная дата платежа'], 
                                                      ['Последняя дата платежа'], 
                                                      ['Есть ошибка в датах?'], 
                                                      ['Нулевая или отрицательная выручка'], 
                                                      ['Общая выручка'], 
                                                      ['Идентификаторы платежа'], 
                                                      ['количество платных продуктов']], 
                                                     index=['Number_of_transactions',
                                                            'Unique_users',
                                                            'Columns_with_gaps',
                                                            'Duplicate_lines',
                                                            'Unique_countries',
                                                            'Min_joined_date',
                                                            'Max_joined_date',
                                                            'Min_paid_date',
                                                            'Max_paid_date',
                                                            'Error_in_dates',
                                                            'Not_positive_revenue',
                                                            'Sum_revenue',
                                                            'Number_of_ids',
                                                            'Number_of_products'])


transactions_data_tale.head(2)

,test,control_1,control_2,explanation
Number_of_transactions,273,1007,328,Размер таблицы
Unique_users,146,193,187,Уникальные пользователи


#### Выводы по сводной таблице transactions 

In [12]:
# Для удобства выводим данные по частям
transactions_data_tale.iloc[:5]

,test,control_1,control_2,explanation
Number_of_transactions,273,1007,328,Размер таблицы
Unique_users,146,193,187,Уникальные пользователи
Columns_with_gaps,[],"[uid, country, joined_at, paid_at, revenue, pa...",[],Колонки с пропусками
Duplicate_lines,7,650,5,Повторяющиеся строки
Unique_countries,17,18,19,Страны


1) **Number_of_transactions** Количество транзакций в первой контрольной группе(1007) значительно больше, чем во второй(328), на 207%.

2) **Unique_users** Уникальных пользователей в каждой таблице меньше числа записей. При этом в тестовой группе мы видим, что их на 27% меньше, чем в среднем по все группам.

3) **Columns_with_gaps** В первой контрольной группе есть несколько столбцов,в которых наблюдаются пропущенные значения, в то время как две другие таблицы пропусков не имеют. Пропуски есть в uid, что само по себе уменьшает количество пригодных для анализ записей

4) **Duplicate_lines** Во всех таблицах есть дубликаты, а это значит, что с точностью до минут повторяется информация о транзацкии. Быть может, конечно, пользователи в некоторых случаях часто совершали некоторые действия.

5) **Unique_countries** В тестовой группе мы наблюдаем всего лишь 17 уникальных стран. Для чистоты эксперимента нам следует:
а) оставить страны, которые наблюдаются во всех 3 группах, б) в таблицах users оставить записи с набором стран из подпункта а.

In [13]:
transactions_data_tale.iloc[5:10]

,test,control_1,control_2,explanation
Min_joined_date,2017-01-11,2015-01-19,2017-01-11,Минимальная дата регистрации
Max_joined_date,2017-10-31,2017-10-31,2017-10-31,Последняя дата регистрации
Min_paid_date,2017-01-11,2016-12-14,2017-01-11,Минимальная дата платежа
Max_paid_date,2017-12-11,2017-12-11,2017-12-11,Последняя дата платежа
Error_in_dates,True,True,True,Есть ошибка в датах?


6) **Min_joined_date** Вновь во второй контрольной группе мы наблюдаем, что первая дата регистрации отличается на 2 года от дат в тестовой и второй контрольной группах. Кроме этого периоды дат регистрации совпадают полностью во всех группах.

7) **Min_paid_date** Аналогичная ситуация с датами оплат.

8) Пожалуй, мы можем утверждать, что новые пользователи - это те, кто зарегистрировался в период 2017-01-11 - 2017-10-31 и при этом совершал оплату в период 2017-01-11 - 2017-12-11

9) Колонка **error_in_dates** говорит нам о том, есть ли в сведениях о датах логическая ошибка, когда оплата происходила раньше, чем регистрация. Ошибка наблюдается повсеместно, но при этом максимальные даты регистрации везде одинаковы и везде не превосходят максимальные даты оплат. 

In [14]:
transactions_data_tale.iloc[10:]

,test,control_1,control_2,explanation
Not_positive_revenue,False,False,False,Нулевая или отрицательная выручка
Sum_revenue,2344901,2.61231e+06,1920438,Общая выручка
Number_of_ids,6,7,6,Идентификаторы платежа
Number_of_products,4,4,4,количество платных продуктов


9) Все значения revenue в записях строго положительны

10) **Number_of_ids** Мы не наблюдаем, что в тестовой группе есть на два способа оплаты больше, чем в других; при этом в первой контрольной группе способов оплаты даже на один больше. Значит будем иначе искать те платформы, на которых теперь другая стоимость оплаты.

11) Количество продуктов везде одинаково.

In [15]:
# Сравним общие выручки по группам из файлов users(строка 0) и transactions(1)
a = users_data_tale.loc[users_data_tale.index == 'Sum_revenue']
b = transactions_data_tale.loc[transactions_data_tale.index == 'Sum_revenue']

pd.concat([a, b], ignore_index=True)

,test,control_1,control_2,explanation
0,2300818,2581267,1920438,Общая выручка
1,2344901,2.61231e+06,1920438,Общая выручка


Общие суммы выручки совпадают лишь в случае 2 контрольной группы.

In [16]:
# Соберем информацию о типах данных в таблицах о транзакициях в одну таблицу

transactions_dtypes = transactions_test.dtypes.to_frame().rename(columns={0: 'test'})
transactions_dtypes['control_1'] = transactions_control_1.dtypes
transactions_dtypes['control_2'] = transactions_control_2.dtypes

transactions_dtypes

,test,control_1,control_2
uid,int64,float64,int64
country,object,object,object
joined_at,datetime64[ns],datetime64[ns],datetime64[ns]
paid_at,datetime64[ns],datetime64[ns],datetime64[ns]
revenue,int64,float64,int64
payment_id,int64,float64,int64
from_page,object,object,object
product_type,object,object,object


Есть изменения в колонках uid, revenue и payment_id. Подобная ситуация в числовых колонках возникает, если в них есть пустые значения. При необходимости, приведем указанные колонки к нужному нам типу данных.

### Предварительная обработка данных. Users

In [17]:
users_test.isna().sum()

uid                    0
age                    0
attraction_coeff       0
coins               4259
country                0
visit_days          1330
gender                 0
age_filter_start       0
age_filter_end         0
views_count            0
was_premium         3900
is_premium          4151
total_revenue          0
dtype: int64

В файлах users мы наблюдали одни и те же 4 колонки с пропущенными значениями. Количество пропусков велико, мы не можем их удалить.

In [18]:
users_test.loc[users_test.coins.notna()].head(3)

,uid,age,attraction_coeff,coins,country,visit_days,gender,age_filter_start,age_filter_end,views_count,was_premium,is_premium,total_revenue
35,892333656,50,436,4.0,United States of America,"1,2,3,4",1,47,53,126,NaN,NaN,897
44,891589239,21,303,37.0,Italy,"1,2,3,4,5,6,7,8,9",1,18,24,181,1.0,1.0,37817
100,891343671,26,0,24.0,France,"1,2,3,4,5,6,7,8,9",1,23,35,37,1.0,1.0,21580


Мы видим, что монетки есть и у обладателей премиум, и у тех, у кого нет данных о таком статусе. Будем считать, что если нет записей о монетках, то их число равно 0. 

Проверим, есть ли такие ситуации, когда was_premium не было, но при этом премиум активен сейчас

In [19]:
users_test.query('was_premium != 1 and is_premium == 1').shape[0]

0

In [20]:
users_test.query('was_premium == 1 and is_premium != 1').shape[0]

251

Имеется 251 запись, когда пользователь был премиум, но теперь не является и 0 записей, когда премиум активек, но was_premium пуст. Кажется, система записи работает так, как этого ожидаешь. Будем ставить 0, если пользователь не имеет и/или не имел премиум статуса.

In [21]:
# Посмотрим на записи, где есть пропуски в колонке visit_days
users_test_passes = users_test.loc[users_test.visit_days.isna()]
users_test_passes.head()

,uid,age,attraction_coeff,coins,country,visit_days,gender,age_filter_start,age_filter_end,views_count,was_premium,is_premium,total_revenue
1,892044516,27,0,NaN,Germany,NaN,1,24,30,0,NaN,NaN,0
4,891406035,27,1000,NaN,France,NaN,1,24,30,1,1.0,NaN,0
6,891304281,39,0,NaN,France,NaN,1,36,42,0,NaN,NaN,0
7,892431420,21,909,NaN,Australia,NaN,1,18,24,11,NaN,NaN,0
13,891219699,30,1000,NaN,United States of America,NaN,1,27,33,1,1.0,NaN,0


С visit_days мы видим необычную ситуацию. Пользователь может иметь статус was_premium и при этом данные говорят, что он не заходил в приложение.

In [23]:
"""Оценим, есть ли пользователи, которые расплачивались в приложении,
но при этом у них нет записей о посещении приложения.
Эти пользователи гарантированно посещали приложение"""

passes_pays = users_test_passes.query('total_revenue > 0')
passes_pays.head(1)

,uid,age,attraction_coeff,coins,country,visit_days,gender,age_filter_start,age_filter_end,views_count,was_premium,is_premium,total_revenue
134,891919368,29,250,NaN,France,NaN,1,26,32,4,1.0,1.0,16536


Такие пользователи есть. Посмотрим как количество таких пользователей соотносится со всеми платящими пользователями

In [24]:
a = passes_pays.shape[0]
b = users_test.query('total_revenue > 0').shape[0]

print(f'Мы имеем {a} пользователей, которые якобы не заходили в приложение после регистрации,\n\
но при этом совершали оплату в приложении. Также в файле users_test у нас всего\n\
{b} пользователей, которые совершали оплату. Таким образом {round(a/b * 100)} процентов платящих\n\
пользователей не имеют записи о том, что они заходили в приложение.')

Мы имеем 19 пользователей, которые якобы не заходили в приложение после регистрации,
но при этом совершали оплату в приложении. Также в файле users_test у нас всего
146 пользователей, которые совершали оплату. Таким образом 13 процентов платящих
пользователей не имеют записи о том, что они заходили в приложение.


In [25]:
# Возьмем uid пользователей, которые платили, но не заходили в приложение
no_visit_uid = passes_pays.uid.to_list()

# Посмотрим на информацию о времени регистрации и платежа таких пользователей
no_visit_transactions = transactions_test.query('uid in @no_visit_uid')
no_visit_transactions[['uid', 'joined_at', 'paid_at']].head()

,uid,joined_at,paid_at
24,891786216,2017-10-27 18:20:00,2017-03-11 20:20:00
40,892093275,2017-10-21 09:18:00,2017-10-28 11:18:00
47,891640689,2017-10-30 13:47:00,2017-06-11 17:31:00
92,891904293,2017-10-25 06:16:00,2017-02-11 08:16:00
99,891231273,2017-07-11 03:39:00,2017-11-14 09:47:00


Есть пользователи, у которых есть ошибка, когда дата регистрации больше даты платежа, но есть и пользователи, у которых даты расположены в хронологическом порядке. 

Таким образом, платящие пользователи - единственные с пропусками в visit_days, у кого мы можем надежно установить, что они заходили в приложение. Тем не менее, их количество мало, поэтому мы не будем восстанавливать эти пропущенные значения. И все пропуски заполним нулями.

In [26]:
# Посмотрим распредление пола в таблице
users_test.gender.value_counts()

1    4106
0     202
Name: gender, dtype: int64

Подавляющее большинство пользователи гендера 1, будем считать что это мужчины, следуя статье.

https://en.wikipedia.org/wiki/Tinder_(app)#Users

In [27]:
"""Эта функция позволит нам проделать с файлами users сразу несколько операций:
1) Мы посчитаем для каждого пользователя сколько дней посещал приложение
2) Мы поменяем обозначение пола в соответствующем столбце
3) Создадим столбец, где будет записано что это за группа
4) Заполним пропуски"""

def processing_1(df, group_name):
    
    df['visit_days'] = df['visit_days'].str.split(',').str.len()
    df['gender'] = df.gender.apply(lambda x: 'male' if x == 1 else 'female')
    df['group'] = group_name
    df.fillna(0, inplace=True)
    
    return df

In [28]:
# Обратимся к функции, которая делает ряд преобразований с фалйлами users
users_test = processing_1(users_test, 'test')
users_control_1 = processing_1(users_control_1, 'control_1')
users_control_2 = processing_1(users_control_2, 'control_2')

##### Изменения в users

1) Преобразована колонка visit_days. В каждом столбце теперь стоит количество дней, которое пользователь бывал в приложении.

2) В колонке gender явным образом указан пол пользователей( опираемся на данные, которые предоставил tinder).

3) Появилась колонка group, в которой есть информация о группе

4) Все пропуски заполнены нулями

### Предварительная обработка данных. Transactions 

##### Пропуски 

In [29]:
transactions_control_1.isna().sum()

uid             630
country         630
joined_at       630
paid_at         630
revenue         630
payment_id      630
from_page       630
product_type    630
dtype: int64

Похоже, мы имеем 630 пустых строк. Во всяком случае, если у строки нет uid, она становится бесполезной для нашего анализа. Удалим строки, где пропущен uid

In [30]:
transactions_control_1.dropna(subset=['uid'], inplace=True)
transactions_control_1.isna().sum()
# Теперь все записи с пустыми значениями удалены

uid             0
country         0
joined_at       0
paid_at         0
revenue         0
payment_id      0
from_page       0
product_type    0
dtype: int64

В transactions_control_1 мы имели проблему с датами регистрации и оплаты, которые были раньше чем в двух других таблицах. Возьмем даты из двух других таблиц за эталонные и посмотрим какое количество записей выбиваются из промежутка, в котором проходил эксперимент.

In [31]:
# Записи о регистрации из transactions_control_1, прошедшие раньше начала эксперимента
min_joined_date = transactions_test.joined_at.min().date()
error_joined_date = transactions_control_1.query('joined_at < @min_joined_date')
error_joined_date

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
65,960936960.0,Russian Federation,2015-01-19 11:49:00,2016-12-14 17:30:00,26.0,0.0,refund_VP,other_type
101,960936960.0,Russian Federation,2015-01-19 11:49:00,2016-12-14 17:35:00,637.0,0.0,refung_AP,other_type
224,960936960.0,Russian Federation,2015-01-19 11:49:00,2016-12-14 16:16:00,143.0,0.0,refund_WP,other_type


In [32]:
# Записи о платежах из transactions_control_1, прошедшие раньше начала эксперимента
min_paid_date = transactions_test.paid_at.min().date()
error_paid_date = transactions_control_1.query('paid_at < @min_paid_date')
error_paid_date

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
65,960936960.0,Russian Federation,2015-01-19 11:49:00,2016-12-14 17:30:00,26.0,0.0,refund_VP,other_type
101,960936960.0,Russian Federation,2015-01-19 11:49:00,2016-12-14 17:35:00,637.0,0.0,refung_AP,other_type
224,960936960.0,Russian Federation,2015-01-19 11:49:00,2016-12-14 16:16:00,143.0,0.0,refund_WP,other_type


Три повторяющихся строки одного пользователя. Эти записи нарушают чистоту эксперимента, поэтому мы их удалим.

In [33]:
for_removing = error_joined_date.uid.to_list()
transactions_control_1 = transactions_control_1.query('uid != @for_removing')

Отберем страны, которые есть во всех трех таблицах.

In [34]:
# Воспользуемся встроенной структурой данных - множеством, которое содержит лишь уникальные элементы.
a_1 = set(transactions_test.country)
b_1 = set(transactions_control_1.country)
c_1 = set(transactions_control_2.country)

# Пересечение множеств оставит страны, которые присутствуют в каждой из таблиц
required_countries = list(a_1 & b_1 & c_1)


# Отберем во всех файлах записи, страны в которых есть в каждом из файлов.
users_test = users_test.query('country in @required_countries')
users_control_1 = users_control_1.query('country in @required_countries')
users_control_2 = users_control_2.query('country in @required_countries')

transactions_test = transactions_test.query('country in @required_countries')
transactions_control_1 = transactions_control_1.query('country in @required_countries')
transactions_control_2 = transactions_control_2.query('country in @required_countries')

# Выведем количество стран, которое теперь есть в файлах
len(required_countries)

15

##### Логическая ошибка в датах

In [35]:
# Отберем даты начала и конца эксперимента
min_date = transactions_test.joined_at.min().date()
max_date = transactions_test.paid_at.max().date()

print(f'Все записи эксперимента лежат в интервале от {min_date} до {max_date}')

Все записи эксперимента лежат в интервале от 2017-01-11 до 2017-12-11


In [36]:
# Посмотрим как выглядят ошибочные записи
error_date = transactions_test.query('joined_at > paid_at')[['joined_at', 'paid_at']]
error_date.head()

,joined_at,paid_at
11,2017-10-26 02:55:00,2017-10-11 19:00:00
20,2017-10-27 08:56:00,2017-03-11 10:57:00
24,2017-10-27 18:20:00,2017-03-11 20:20:00
25,2017-10-21 23:15:00,2017-08-11 01:56:00
36,2017-10-31 14:16:00,2017-09-11 15:44:00


1) Даты не выходят за границы эксперимента

2) У дат оплаты есть особенность. Везде стоит 11 день месяца. Проверим какие дни месяца и месяца есть в записях

In [37]:
# Дни месяца в датах оплаты
set(error_date.paid_at.dt.day)

{11}

In [38]:
# Месяца там же
set(error_date.paid_at.dt.month)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

Мы видим, что во всех ошибочных записях день месяца 11, а даты месяцов различаются.

Посмотрим какие месяцы при в записях о регистрации

In [39]:
set(error_date.joined_at.dt.month)

{10}

Итак, судя по всему произошла ошибка, в результате которой месяц и день записывались на место друг друга. Это было в период 2017-11-01 - 2017-11-10. Посмотрим так ли выглядят данные в двух других файлах

In [40]:
error_date_ctrl_1 = transactions_control_1.query('joined_at > paid_at')[['joined_at', 'paid_at']]
error_date_ctrl_2 = transactions_control_2.query('joined_at > paid_at')[['joined_at', 'paid_at']]

print(f'Файл transactions_control_1\n\
Дни месяца в датах оплаты {set(error_date_ctrl_1.paid_at.dt.day)}\n\
Месяц регистрации {set(error_date_ctrl_1.joined_at.dt.month)}\n')

print(f'Файл transactions_control_2\n\
Дни месяца в датах оплаты {set(error_date_ctrl_2.paid_at.dt.day)}\n\
Месяц регистрации {set(error_date_ctrl_2.joined_at.dt.month)}')

Файл transactions_control_1
Дни месяца в датах оплаты {11}
Месяц регистрации {10}

Файл transactions_control_2
Дни месяца в датах оплаты {11}
Месяц регистрации {10}


Ситуация повторяется в оставшихся файлах. Перезаписывать даты я не буду, так как не планирую вести их анализ

##### 2 новые платежные системы

In [41]:
# Найдем уникальные идентификаторы в каждом из файлов
a = set(transactions_test.payment_id)
b = set(transactions_control_1.payment_id.astype(int))
c = set(transactions_control_2.payment_id)

In [42]:
# Найдем разность множеств и тем самым увидим элементы, которые есть только в тестовой группе
a - b - c

set()

Это странно. В тестовой группе нет уникальных платежных систем. Или payment_id не является указателем на платежную систему. Или платежные системы были одинаковы для всех 3 групп, но в тестовой цена менялась при покупке премиум.

##### Дубли и различие в суммах

Мы уже провели некоторое изменение данных. Детально посмотрим что изменилось и как выглядят выручки в таблицах

In [43]:
def after_processing(df_u, df_tr, group_name):
    users_uids = set(df_u.uid)
    trans_uids = set(df_tr.uid)
    
    result = pd.DataFrame([[df_u.total_revenue.sum()],
                           [df_tr.drop_duplicates().revenue.sum()],
                           [df_tr.revenue.sum()],
                           [df_u.uid.nunique()],
                           [df_u.shape[0]],
                           [df_tr.uid.nunique()],
                           [df_tr.shape[0]],
                           [len(trans_uids - users_uids)]],
                          index=['Users_sum_revenue', 
                                 'Transactions_rev_without_dupl', 
                                 'Transactions_sum_revenue', 
                                 'Users_uniq_uid', 
                                 'Users_table_size', 
                                 'Transactions_uniq_uid', 
                                 'Transactions_table_size', 
                                 'Uid_only_in_trans-s'], 
                          columns=[group_name])
                                                         
    return result

In [44]:
# Воспользуемся функцией и объединим результаты
after_processing_table = after_processing(users_test, transactions_test, 'test')
after_processing_table['control_1'] = after_processing(users_control_1, transactions_control_1, 'control_1')
after_processing_table['control_2'] = after_processing(users_control_2, transactions_control_2, 'control_2')
after_processing_table['explanation'] = pd.DataFrame([['Общая выручка в файлах users'],
                                                      ['Выручка в trans-s без дубликатов'],
                                                     ['Общая выручка в файлах trans-s'],
                                                     ['Уник. польз-ли в файлах users'],
                                                     ['Размер т. users после обработки'],
                                                      ['Уник. пользователи в файлах trans-s'],
                                                      ['Размер trans-s после обработки'],
                                                      ['Пользователи в trans-s, которых нет в users']],
                                                     index=['Users_sum_revenue',
                                                            'Transactions_rev_without_dupl',
                                                            'Transactions_sum_revenue',
                                                            'Users_uniq_uid',
                                                            'Users_table_size',
                                                            'Transactions_uniq_uid',
                                                            'Transactions_table_size',
                                                            'Uid_only_in_trans-s'])

after_processing_table

,test,control_1,control_2,explanation
Users_sum_revenue,2284503,2552186.0,1832818,Общая выручка в файлах users
Transactions_rev_without_dupl,2316847,2464943.0,1820377,Выручка в trans-s без дубликатов
Transactions_sum_revenue,2328586,2582450.0,1832818,Общая выручка в файлах trans-s
Users_uniq_uid,4142,4189.0,4100,Уник. польз-ли в файлах users
Users_table_size,4142,4189.0,4100,Размер т. users после обработки
Transactions_uniq_uid,144,188.0,178,Уник. пользователи в файлах trans-s
Transactions_table_size,267,368.0,310,Размер trans-s после обработки
Uid_only_in_trans-s,0,0.0,0,"Пользователи в trans-s, которых нет в users"


1) В первых двух группах в файлах users общая выручка меньше, чем в transactions. В третьей группе выручка изначально равная, но, при удалении дубликатов, выручка в transactions становится меньше. Это сигнал в пользу того, что дубликаты - это на самом деле лишь несколько одинаковых покупок пользователя в течение 1 минуты.

2) Размеры users стали, очевидно, меньше и разница в пользователях ужалась.

3) Последняя строчка таблицы говорит нам, что в transactions нет таких пользователей, которые бы не встречались в users. Тогда, отложив пока в сторону мысли о дубликатах, можем дать несовпадению выручек следующее объяснение. В total_revenue у некоторых пользователей записано ошибочное число.

##### Дубликаты

In [45]:
# выведем дубли для первой группы
transactions_test.loc[transactions_test.duplicated()].head(3)

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
55,892236423,United States of America,2017-10-18 01:09:00,2017-10-23 00:15:00,3783,67,menu,other_type
96,892050108,Israel,2017-10-22 03:02:00,2017-10-23 02:53:00,1261,19,empty_likes,coins
127,892236423,United States of America,2017-10-18 01:09:00,2017-10-27 22:38:00,3783,67,menu,other_type


Из всех колонок, которые здесь присутствуют посмотрим на колонку product_type. Остальные, на мой взгляд, не привносят информацию о том являются ли дубли ошибкой или это реальное поведение пользователя.

In [46]:
# Функция выводит тип продукта в дублируемых строчках
def product_type_dupl(df):
    result = df.loc[df.duplicated()].product_type.unique()
    return result

In [47]:
a = product_type_dupl(transactions_test)
b = product_type_dupl(transactions_control_1)
c = product_type_dupl(transactions_control_2)
print(f'Типы продукта в transactions_test      {a}\n\
типы продукта в transactions_control_1 {b}\n\
типы продукта в transactions_control_2 {c}')

Типы продукта в transactions_test      ['other_type' 'coins']
типы продукта в transactions_control_1 ['other_type' 'coins']
типы продукта в transactions_control_2 ['coins' 'other_type']


Во всех 3 группах дублируются операции покупки монеток и "другое". Если бы это была покупка подписки, можно было бы уверенно сказать, что это ошибка. Вполне можно представить ситуацию, что люди докупают монетки, даже на одинаковую сумму оба раза, в том числе и по ошибке. Но у нас нет информации о возвратах, а также нет других опознавательных факторов, чтобы оценить это. 

Мы не можем надежно показать ни то, что это ошибка, ни то, что это логично вписывающееся поведение. Поэтому, учитывая небольшое количество дублей, мы их удалим. Иначе нам предстоит проверять гипотезы на таблицах и с дублями и без.

In [48]:
# Удаление дубликатов
transactions_test.drop_duplicates(inplace=True)
transactions_control_1.drop_duplicates(inplace=True)
transactions_control_2.drop_duplicates(inplace=True)

##### Посмотрим стоимость подписки

In [49]:
# Функция выводит для 3 групп ценник подписки или медианную цену в зависимости от страны
def change_cost(df, func='unique'):
    if func == "median": 
        result = df.query('product_type == "premium_no_trial"') \
            .groupby('country') \
            .agg(name=('revenue', 'median'))
        
    else:
        result = df.query('product_type == "premium_no_trial"') \
            .groupby('country') \
            .agg(name=('revenue', 'unique'))
    
    return result

In [50]:
prices = change_cost(transactions_test).rename(columns={'name': 'test'})
prices['control_1'] = change_cost(transactions_control_1)
prices['control_2'] = change_cost(transactions_control_2)

prices.head(3)

,test,control_1,control_2
country,,,
Argentina,"[5330, 12597, 6292]","[5278.0, 6292.0]","[6292, 5278, 5265, 10413]"
Australia,[12597],[3588.0],[4719]
Belgium,[12597],[3042.0],"[10608, 6292, 6461, 3042]"


В рамках одной страны есть несколько стоимостей подписки. Это значит, что четких различий мы не увидим. Более того, нам нужно принять два решения:

1) В задании сказано, что стоимость подписки изменилась, при этом не сказано в большую или меньшую сторону. А значит в некоторых странах цена могла уменьшиться, в а в других вырасти. По всей видимости, нам придется взять за основу лишь однонаправленные изменения цены.

2) Все цены будут либо возросшими, либо уменьшенными.

Кроме этого, стоит оценить есть ли в распределении стоимости подписки выбросы, чтобы понять какую меру центральной тенденции использовать для сравнения

In [51]:
# Распределение стран по выручке
transactions_test.query('product_type == "premium_no_trial"').groupby('country') \
    .revenue.describe() \
    .sort_values('mean', ascending=False).head()

,count,mean,std,min,25%,50%,75%,max
country,,,,,,,,
Spain,6.0,24918.833333,43470.135033,5369.0,5629.0,6331.0,11040.25,113477.0
United Arab Emirates,7.0,23422.285714,39741.671692,4914.0,8957.0,9217.0,9217.00,113477.0
Australia,1.0,12597.000000,NaN,12597.0,12597.0,12597.0,12597.00,12597.0
Belgium,1.0,12597.000000,NaN,12597.0,12597.0,12597.0,12597.00,12597.0
Israel,2.0,12597.000000,0.000000,12597.0,12597.0,12597.0,12597.00,12597.0


Обратим внимание на первые две строчки, колонку max. Явно выражены выбросы. Значит для сравнения стоимости будем использовать медиану.

Проверим оба варианта: и уменьшение и увеличение

In [52]:
prices = change_cost(transactions_test, 'median').rename(columns={'name': 'test'})
prices['control_1'] = change_cost(transactions_control_1, 'median')
prices['control_2'] = change_cost(transactions_control_2, 'median')

prices.query('test > control_1 and test > control_2').shape[0]

10

In [53]:
prices.query('test < control_1 and test < control_2').shape[0]

0

В варианте, где уменьшили цену, нет ни одной страны. Соответственно, будем считать, что цены были увеличены

In [54]:
# Список стран, в которых изменилась цена подписки
countries_final_list = prices.query('test > control_1 and test > control_2').index.to_list()

##### После предобработки в transactions

1) Убрали пропуски

2) Убрали записи, которые выходя за даты эксперимента

3) Убрали дубли

4) Не нашли 2 новые платежные системы

5) Нашли в чем заключается ошибка записи дат

6) Отобрали страны, в которых проходил эксперимент

### Объединение users и transactions в один df

In [55]:
# Объединение df
def merge_df(df_u, df_tr):
    new_df = df_u.merge(df_tr, how='left', on=['uid', 'country'])
    return new_df

In [56]:
test_group = merge_df(users_test, transactions_test)
control_1_group = merge_df(users_control_1, transactions_control_1)
control_2_group = merge_df(users_control_2, transactions_control_2)

##### Проверим несовпадение сумм

Именно после объединения у нас появилась простая возможность увидеть записи, где есть несовпадание выручек.

Выручка несовпадала в тестовой и первой контрольной группах

In [57]:
# Несовпадние в test_group
revenue_difference = test_group.groupby('uid', as_index=False) \
    .agg(total_revenue=('total_revenue', 'unique'), sum_rev=('revenue', 'sum')) \
    .query('total_revenue < sum_rev')
revenue_difference.head()

,uid,total_revenue,sum_rev
3629,892275462,[14885],58968.0


Получили одного пользователя, у которого в users, судя по всему, неправильно записана выручка

In [58]:
# выведем записи пользователя
diff_uids = revenue_difference.uid.to_list()
test_group.query('uid in @diff_uids')[['uid', 'was_premium', 'is_premium', 
                                       'total_revenue', 'revenue', 'product_type']]

,uid,was_premium,is_premium,total_revenue,revenue,product_type
2066,892275462,1.0,1.0,14885,44083.0,other_type
2067,892275462,1.0,1.0,14885,14885.0,other_type


Обратим внимание на колонку revenue, первую строку. Найдено отличие(**44083.0**). Добавим его к общей выручке users и выясним как она будет соотноситься с выручкой в transactions 

Для напоминания выведем таблицу, по которой мы сравнивали выручки. Напомню, что на данном этапе дубли удалены, но наша задача лишь выяснить как соотносятся выручки с учетом ошибочных записей, поэтому старая таблица подойдет.

In [59]:
after_processing_table.iloc[0:3]

,test,control_1,control_2,explanation
Users_sum_revenue,2284503,2552186.0,1832818,Общая выручка в файлах users
Transactions_rev_without_dupl,2316847,2464943.0,1820377,Выручка в trans-s без дубликатов
Transactions_sum_revenue,2328586,2582450.0,1832818,Общая выручка в файлах trans-s


In [60]:
# Возьмем колонку тест. В записях была ошибка 44083.0 
2328586 - 2284503

44083

In [61]:
# 
after_processing_table.T.Users_sum_revenue[0] + 44083.0 == after_processing_table.T.Transactions_sum_revenue[0]

True

Сделаем то же для группы control_1

In [62]:
revenue_difference_1 = control_1_group.groupby('uid', as_index=False) \
    .agg(total_revenue=('total_revenue', 'unique'), sum_rev=('revenue', 'sum')) \
    .query('total_revenue < sum_rev')

diff_uids = revenue_difference_1.uid.to_list()
control_1_group.query('uid in @diff_uids')[['uid', 'was_premium', 'is_premium', 
                                            'total_revenue', 'revenue', 'product_type']]

,uid,was_premium,is_premium,total_revenue,revenue,product_type
3391,892355163,1.0,0.0,0,30264.0,other_type


In [63]:
after_processing_table.T.Users_sum_revenue[1] + 30264.0 == after_processing_table.T.Transactions_sum_revenue[1]

True

В каждой из групп(test, control_1) было найдено по 1 пользователю, для которого было неправильно записано значение общей выручки.

Оставим в объединенных df колонку revenue, а total_revenue можно удалить.

In [64]:
"""
1) Удаление ненужных колонок
2) Заполнение пропусков в revenue(образовались при объединении таблиц)
3) Оставили только страны, в которых изменена стоимость премиум-подписки
"""
def processing_2(df, country_list):
    
    df = df.drop(columns=['age_filter_start', 'age_filter_end', 'joined_at',
                          'paid_at', 'total_revenue', 'payment_id', 'from_page'])
    df.revenue.fillna(0, inplace=True)
    df = df.query('country in @country_list')
    
    return df

In [65]:
test_group = processing_2(test_group, countries_final_list)
control_1_group = processing_2(control_1_group, countries_final_list)
control_2_group = processing_2(control_2_group, countries_final_list)

#### Проверка независимости групп

Одним из обязательных условий в стат. тестах является требование независимости выборок. Проверим, что 3 группы содержат уникальных относительно друг друга пользователей

In [66]:
a = set(test_group.uid)
b = set(control_1_group.uid)
c = set(control_2_group.uid)
a & b & c

set()

#### Вывод по разделу
1) Есть 3 объединенных датафрейма для каждой экспериментальной группы

2) Выяснена и учтена причина несовпадения выручек

3) Все три группы являются независимыми

## Сравнение групп 

Итак, мы имеем 3 группы. Но прежде чем приступить к тестам стоит проверить, что группы идентичны по тем показателям, которые не являются целевыми.

Проверку будем производить не на глаз, а спомощью статистики. Будем рассматривать 2 гипотезы:

Н<sub>0</sub>: распределение пользователей по характеристике/метрике не отличается в группах.

Н<sub>1</sub>: распределение пользователей отличается.

$𝛼 = 0.05$

В данном случае, мы хотели бы видеть, что распределения не отличаются по сравнивыемым характеристикам. Это бы показало, что группы распределены "равномерно"

In [67]:
# Для удобства объединим все 3 df в один.
merged_df = pd.concat([test_group, control_1_group, control_2_group], ignore_index=True)
merged_df.shape[0]

10691

##### Возраст

Первая характеристика возраст. Можно считать ее непрерывно распределенной. Здесь можно сравнить средние значения, но групп 3, поэтому воспользуемся дисперсионным анализом. Ранее показана независимость групп и это условие будет соблюдено для всех будущих тестов. 

В случае же дисперсионного анализа важны: нормальность распределения выборок и небольшая разница дисперсий внутри них(гомогенность дисперсий). Сейчас выборки имеют размеры более 3 тычяч пользователей каждая, это позволяет утверждать, что они рапределены нормально, согласно ЦПТ.

А гомогенность дисперсий нам предстоит проверить

In [68]:
# Отбираем df для проверки 
merged_age = merged_df.groupby(['uid', 'group'], as_index=False) \
    .agg(age=('age', 'first'))

merged_age.head()

,uid,group,age
0,891050916,control_2,34
1,891050997,control_1,38
2,891051075,control_2,39
3,891051147,test,21
4,891051960,control_2,27


In [69]:
# Проверка на равенство дисперсий
pg.homoscedasticity(data=merged_age, dv='age', group='group')

,W,pval,equal_var
levene,0.402615,0.66858,True


p-value больше порогового значения 𝛼 и это не позволяет нам отвергнуть нулевую гипотезу о равенстве дисперсий. Кроме того сам вывод функции(последняя колонка) говорит нам о равенстве дисперсий 3 групп

In [70]:
# Дисперсионный анализ
pg.anova(data=merged_age, dv='age', between='group')

,Source,ddof1,ddof2,F,p-unc,np2
0,group,2,10341,0.877626,0.4158,0.00017


p-value вновь выше порогового значения. Это означает для нас, что группы не отличаются по возрасту.

##### Коэффициент привлекательности

Данную характеристику тоже можно считать непрерывно распределенной. Поэтому проверки далее будут схожи предыдущему пункту

In [71]:
merged_attr_coeff = merged_df.groupby(['uid', 'group'], as_index=False) \
    .agg(attraction_coeff=('attraction_coeff', 'first'))

pg.homoscedasticity(data=merged_attr_coeff, dv='attraction_coeff', group='group')

,W,pval,equal_var
levene,1.443397,0.236172,True


In [72]:
pg.anova(data=merged_attr_coeff, dv='attraction_coeff', between='group')

,Source,ddof1,ddof2,F,p-unc,np2
0,group,2,10341,1.280151,0.278039,0.000248


Дисперсии равны и мы смело проверям группы с помощью дисперсионного анализа, который показывает нам, что группы не различаются по коэффициенту привлекательности

##### Coins

Аналогично считаем ее непрерывно распределнной и проверяем как и выше.

In [73]:
merged_coins = merged_df.groupby(['uid', 'group'], as_index=False) \
    .agg(coins=('coins', 'first'))

pg.homoscedasticity(data=merged_coins, dv='coins', group='group')

,W,pval,equal_var
levene,0.832042,0.435189,True


In [74]:
pg.anova(data=merged_coins, dv='coins', between='group')

,Source,ddof1,ddof2,F,p-unc,np2
0,group,2,10341,0.832042,0.435189,0.000161


Вновь дисперсионный анализ говорит нам, что мы можем считать наши группы одинаковыми по распределению среднего количества монеток в них

##### Страны

Страны - это характеристика, которая является категориальной. В данном случае мы будем проверять, что количество пользователей по странам распределено равномерно в каждой группе( соблюдается одинаковая пропорция)

In [75]:
merged_countries = merged_df.groupby(['uid', 'group'], as_index=False) \
    .agg(country=('country', 'first'))

In [76]:
# Построим таблицу, которая покажет нам количество пользоватлей в отобранных странах
pd.crosstab(merged_countries.country, merged_countries.group)

group,control_1,control_2,test
country,,,
Argentina,389,397,396
Australia,50,51,62
Belgium,108,90,92
Chile,287,286,272
France,370,376,359
Israel,109,78,106
Italy,338,293,323
United Arab Emirates,207,211,226
United Kingdom (Great Britain),146,146,169


Тест хи-квадрат, который применяется ниже, повзолит проверить является ли наше распределение пользователей равномерным в зависимости от страны

In [77]:
exp, obs, stats = pg.chi2_independence(data=merged_countries, x="group", y="country")
stats

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,17.070640,18.0,0.518253,0.028725,0.355872
1,cressie-read,0.666667,17.108442,18.0,0.515661,0.028757,0.356729
2,log-likelihood,0.000000,17.197592,18.0,0.509562,0.028832,0.358753
3,freeman-tukey,-0.500000,17.276748,18.0,0.504161,0.028898,0.360551
4,mod-log-likelihood,-1.000000,17.366903,18.0,0.498030,0.028974,0.362599
5,neyman,-2.000000,17.582077,18.0,0.483486,0.029153,0.367490


Здесь функция выводит несколько значений p-value, но все они больше порогового. Это в свою оцередь значит, мы не можем отвергнуть нулевую гипотезу и распределение по странам равномерно.

##### Количество дней в приложении

Эту характеристику будем считать непрерывной и сравнивать будем средние

In [78]:
merged_visit_days = merged_df.groupby(['uid', 'group'], as_index=False) \
    .agg(visit_days=('visit_days', 'first'))

pg.homoscedasticity(data=merged_visit_days, dv='visit_days', group='group')

,W,pval,equal_var
levene,0.964325,0.381275,True


In [79]:
pg.anova(data=merged_visit_days, dv='visit_days', between='group')

,Source,ddof1,ddof2,F,p-unc,np2
0,group,2,10341,1.528432,0.216924,0.000296


Пользователи во всех группах в среднем проводили одинаковое количество дней в приложении

##### Пол

Эта характеристика имеет две градации и как и страны является категориальной. Будем использовать Хи-квадрат

In [80]:
merged_gender = merged_df.groupby(['uid', 'group'], as_index=False) \
    .agg(gender=('gender', 'first'))

pd.crosstab(merged_gender.gender, merged_gender.group)

group,control_1,control_2,test
gender,,,
female,169,175,175
male,3306,3239,3280


In [81]:
exp, obs, stats = pg.chi2_independence(data=merged_gender, x="group", y="gender")
stats

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,0.274060,2.0,0.871944,0.005147,0.071194
1,cressie-read,0.666667,0.274419,2.0,0.871788,0.005151,0.071222
2,log-likelihood,0.000000,0.275146,2.0,0.871471,0.005157,0.071280
3,freeman-tukey,-0.500000,0.275701,2.0,0.871229,0.005163,0.071324
4,mod-log-likelihood,-1.000000,0.276264,2.0,0.870984,0.005168,0.071369
5,neyman,-2.000000,0.277417,2.0,0.870482,0.005179,0.071461


По полу пользователи распределены равномерно

##### views_count

Считаем эту величину непрерывно распределенной, сравниваем средние

In [82]:
merged_views_count = merged_df.groupby(['uid', 'group'], as_index=False) \
    .agg(views_count=('views_count', 'first'))

pg.homoscedasticity(data=merged_views_count, dv='views_count', group='group')

,W,pval,equal_var
levene,0.156712,0.854952,True


In [83]:
pg.anova(data=merged_views_count, dv='views_count', between='group')

,Source,ddof1,ddof2,F,p-unc,np2
0,group,2,10341,0.162989,0.849603,0.000032


И по этому показателю пользователи в среднем не отличаются

##### Выручка заработанная не от премиум-подписки

В приложении выручка суммируется из 4 продуктов. Проверим различается ли суммарная выручка от всех продуктов кроме премиум-подписки. Это денежная величина - мы считаем ее непрерывно распределенной. Будем проверять средние

In [84]:
merged_revenue = merged_df.query('product_type != "premium_no_trial" and revenue > 0') \
    .groupby(['uid', 'group'], as_index=False) \
    .agg(revenue=('revenue', 'sum'))

merged_revenue.group.value_counts()

control_1    116
control_2    102
test          80
Name: group, dtype: int64

Количество пользователей все еще достаточно велико(>30), условие нормальности распределения сохраняется

In [85]:
pg.homoscedasticity(data=merged_revenue, dv='revenue', group='group')

,W,pval,equal_var
levene,0.579443,0.560846,True


In [86]:
pg.anova(data=merged_revenue, dv='revenue', between='group')

,Source,ddof1,ddof2,F,p-unc,np2
0,group,2,295,2.016094,0.135006,0.013484


Выручка не от премиум-подписки в группах в среднем равная

#####  выводы

По всем характеристикам за исключением целевой(стоимости премиум-подписки) пользователи в группах не различаются

### A/A тест двух контрольных групп

Если контрольные группы не различаются целевым метрикам - это будет значить, что группы были разбиты равномерно во всем. А это в свою очередь говорит о том, что тест организован корректно, согласно методологии.

#####  Целевые метрики

Я не буду менять те метрики, которые выбрал изначально( доля премиум-подписок относительно всех пользователей, выручка на пользователя, выручка владельца премиум-подписки)

##### Доля платных премиум от общего количества пользователей в группе

In [87]:
# Добавим новую колонку, сообщающую нам является ли пользователь обладателем премиум-подписки
merged_df['premium_no_trial'] = merged_df['product_type'].apply(lambda x: 'yes' if x == "premium_no_trial" else 'no')

# Две группы для сравнения
groups = ['control_1', 'control_2']

Данная метрика является категориальной и проверять мы ее будем с помощью теста Хи-квадрат

In [88]:
AA_first_metric = merged_df.query('group in @groups') \
    .groupby(['uid', 'group'], as_index=False) \
    .agg(premium_no_trial=('premium_no_trial', 'first'))

AA_first_metric.head()

,uid,group,premium_no_trial
0,891050916,control_2,no
1,891050997,control_1,no
2,891051075,control_2,no
3,891051960,control_2,no
4,891052698,control_1,no


In [89]:
pd.crosstab(AA_first_metric.premium_no_trial, AA_first_metric.group)

group,control_1,control_2
premium_no_trial,,
no,3422,3353
yes,53,61


In [90]:
exp, obs, stats = pg.chi2_independence(data=AA_first_metric, x="group", y="premium_no_trial")
stats

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,0.572242,1.0,0.449369,0.009114,0.117692
1,cressie-read,0.666667,0.572267,1.0,0.449359,0.009114,0.117695
2,log-likelihood,0.000000,0.572472,1.0,0.449278,0.009116,0.117719
3,freeman-tukey,-0.500000,0.572761,1.0,0.449164,0.009118,0.117754
4,mod-log-likelihood,-1.000000,0.573167,1.0,0.449003,0.009121,0.117804
5,neyman,-2.000000,0.574328,1.0,0.448544,0.009131,0.117944


Значение p-value больше чем 0.05, значит статистически значимых различий мы не обнаружили. В контрольных группах равное количество обладателей премиум-подписок

##### arpu

Здесь мы будем считать общую выручку( в том числе и не отпремиум-подписки), деленную на всех пользователей в группе. 
Общая выручка = выручка от премиум + остальная выручка. Мы показали выше, что остальная выручка не отличается в группах. Можно сказать, что мы сравниваем выручку от премиум + константу. Поэтому такая метрика для оценивания изменений кореектна

Самая метрика непрерывная. Будем сравнивать средние значения и использовать будем t-тест, так как групп 2 и сам тест в данном случае дает более точные результаты

In [91]:
AA_second_metric = merged_df.query('group in @groups') \
    .groupby(['uid', 'group'], as_index=False) \
    .agg(sum_rev=('revenue', 'sum'))

pg.homoscedasticity(data=AA_second_metric, dv='sum_rev', group='group')

,W,pval,equal_var
levene,1.790747,0.180879,True


In [92]:
x = AA_second_metric.query('group == "control_1"').sum_rev
y = AA_second_metric.query('group == "control_2"').sum_rev

pg.ttest(x, y)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.347441,4341.249212,two-sided,0.177908,"[-82.77, 446.63]",0.032247,0.067,0.26747


Мы не можем отклонить нулевую гипотезу, следовательно, по этому показателю контольные группы распределены равномерно

#### arppu 

метрика представляет из себя величину, которая показывает какая выручка в среднем приходится на покупателя премиум-подписки. Величина непрерывная. Будем использовать t-тест

In [93]:
AA_third_metric = merged_df.query('group in @groups and product_type == "premium_no_trial"') \
    .groupby(['uid', 'group'], as_index=False) \
    .agg(sum_rev=('revenue', 'sum'))

pg.homoscedasticity(data=AA_third_metric, dv='sum_rev', group='group')

,W,pval,equal_var
levene,0.506432,0.477757,True


In [94]:
x = AA_third_metric.query('group == "control_1"').sum_rev
y = AA_third_metric.query('group == "control_2"').sum_rev

pg.ttest(x, y)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.48552,151.685897,two-sided,0.628008,"[-2281.5, 3768.15]",0.077419,0.192,0.077015


Группы не различаются по этой метрике

Ни по одной из 3 целевых метрик контрольные группы не отличаются. Теперь мы можем их объединить и объединенную группу сравнивать с тестовой по этим же метрикам.

##### Объединим группы

In [95]:
merged_df['group'] = merged_df['group'].apply(lambda x: 'B' if x == "test" else 'A')
merged_df.group.value_counts()

A    7138
B    3553
Name: group, dtype: int64

### A/B тест

Сравниваем объединенную группу A с тестовой группой B по тем же целевым метрикам, теми же тестами, если условия применения тестов удовлетворены

##### Конверсия 

In [96]:
AB_first_metric = merged_df \
    .groupby(['uid', 'group'], as_index=False) \
    .agg(premium_no_trial=('premium_no_trial', 'first'))

pd.crosstab(AB_first_metric.premium_no_trial, AB_first_metric.group)

group,A,B
premium_no_trial,,
no,6775,3416
yes,114,39


In [97]:
exp, obs, stats = pg.chi2_independence(data=AB_first_metric, x="group", y="premium_no_trial")
stats

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,4.015447,1.0,0.045085,0.019703,0.517542
1,cressie-read,0.666667,4.071433,1.0,0.043614,0.019839,0.523087
2,log-likelihood,0.000000,4.193623,1.0,0.040576,0.020135,0.535041
3,freeman-tukey,-0.500000,4.294834,1.0,0.038228,0.020376,0.544788
4,mod-log-likelihood,-1.000000,4.404898,1.0,0.035836,0.020636,0.555229
5,neyman,-2.000000,4.654228,1.0,0.030977,0.021212,0.578263


p-value меньше порогового значения. Значит мы может отклонить нулевую гипотезу и считать, что наши группы различаются по этому показателю. А именно, в тестовой группе стало меньше премиум-подписок в сравнении с контрольной

#### ARPU 

In [98]:
AB_second_metric = merged_df \
    .groupby(['uid', 'group'], as_index=False) \
    .agg(sum_rev=('revenue', 'sum'))

pg.homoscedasticity(data=AB_second_metric, dv='sum_rev', group='group')

,W,pval,equal_var
levene,0.003039,0.956035,True


In [99]:
x = AB_second_metric.query('group == "A"').sum_rev
y = AB_second_metric.query('group == "B"').sum_rev

pg.ttest(x, y)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.060593,8862.700002,two-sided,0.951685,"[-199.66, 187.69]",0.001149,0.024,0.050348


А вот по метрике ARPU наши группы не различаются. Это значит, что наша выручка, как бизнеса в пересчете на одного пользователя не изменилась

#### ARPPU 

In [100]:
AB_third_metric = merged_df.query('product_type == "premium_no_trial"') \
    .groupby(['uid', 'group'], as_index=False) \
    .agg(sum_rev=('revenue', 'sum'))

pg.homoscedasticity(data=AB_third_metric, dv='sum_rev', group='group')

,W,pval,equal_var
levene,1.656367,0.199525,True


In [101]:
x = AB_third_metric.query('group == "B"').sum_rev
y = AB_third_metric.query('group == "A"').sum_rev

pg.ttest(x, y)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.884713,67.167479,two-sided,0.063796,"[-242.38, 8458.17]",0.368894,0.875,0.637366


Здесь мы тоже не можем отклонить нулевую гипотезу. Группы не различаются по выручке принесенной нам владельцами премиум-подписок

## Выводы:

1) Мы можем утверждать, что владельцев премиум-подписок стало статистически меньше

2) Выручка в пересчете на всех пользователей не изменилась

3) Выручка с каждого владельца премиум-подписки не изменилась

Эксперимент стоит считать неуспешным